<a href="https://colab.research.google.com/github/3B032007/2025_ML_hws/blob/main/hw5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. 將存好的模型和訓練權重讀回來

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd '/content/drive/My Drive/Colab Notebooks'

/content/drive/My Drive/Colab Notebooks


In [4]:
from tensorflow.keras.models import model_from_json

In [10]:
model = model_from_json(open('imdb_model_architecture.json').read())
model.load_weights('imdb_model.weights.h5')

In [11]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 32)        │        64,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │        82,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 146,561 (572.50 KB)

 Trainable params: 146,561 (572.50 KB)

 Non-trainable params: 0 (0.00 B)

# 2. 載入自行建立的tokenizer

In [14]:
import pickle
f = open('imdb_tokenizer.pkl', 'rb')
tokens = pickle.load(f)
f.close()

In [15]:
from tensorflow.keras.preprocessing import sequence

# 3. 輸入實際由IMDb取得的影評

In [16]:
input_text= '''
Sure, I'm a huge film snob who (on the surface) only likes artsy-fartsy foreign films from before the 60's, but that hasn't stopped me from loving Disney's Beauty & The Beast; in fact, it's probably my favorite American animated film and is easily Disney's finest work. It's beautiful, it's breathtaking, it's warm, it's hilarious, it's captivating, and, in Disney fashion, it's magical. When I learned that Disney would be remaking their classic films, B&TB was undeniably the best wrapped package. How could they go wrong?
Oh man, they went wrong.
First thing's first: this film is so flat. The directing was dull and uninteresting throughout the entire film and it honestly felt like one of the Twilight sequels...and then I looked it up and found out that, yes, director Bill Condon was the man behind Breaking Dawn parts 1 & 2. Every shot looks bored and uninterested, which contrasts heavily with the original animated film that was constantly popping with vibrancy. The script too is boring because it's almost a complete remake of the original, though I guess most people won't mind that.
Next: the CGI is horrid. Although I didn't care for The Jungle Book from last year, I could at least admit that the CGI was breathtaking. The same cant be said for this film. Characters like Lumière, Cogsworth, Mrs Potts, and most of the cursed appliances have very strange, lifeless faces that are pretty off putting to be looking at for such a long time. All of the sets too look artificial and fake, especially the town towards the beginning. However, the biggest offender is easily and infuriatingly the character that mattered most: The Beast. The CGI on the Beast's face is so distracting that it completely takes you out of the film. His eyes are completely devoid of soul, and his mouth is a gaping video game black hole of fiction. Klaus Kinski looked much better in the Faerie Tale Theatre episode of Beauty & The Beast, and that was a 1984 TV show episode. But do you know why it looked better? Because it was an actual face with actual eyes, not some video game computerized synthetic monstrosity. When will studios learn that practical effects will always top CGI?
Finally: wasted casting. Emma Watson is beautiful, but she's no Belle. She is completely devoid of the warmth and humanity that made the animated Belle so beloved. Instead, she is cold and heartless throughout most of the film. Kevin Kline is 100% wasted and does nothing except look old. Ian McKellan, Ewan McGregor, Emma Thompson, and even Dan Stevens as the Beast are very expendable and could've been played by anyone else. The only good characters are Gaston and LeFou, mostly because they are fun and played by actors who breathe new life into their original shapes. If anything, this film should've been about Gaston and LeFou, but that would never happen because that would mean Disney couldn't cater to blind nostalgic 90's kids.
Overall, this film is a complete bore. It could've been better if even the special effects were good, but the CGI in particular is horrendous. I'm all for Disney remaking their nostalgia- catering 90's films, but they need to be interesting. This film, sadly, is not. Even the Christmas sequel is better than this film because it's at least something.
'''

# 4. 轉換影評文字串為數字串，並截長或補短，使數字串長度為100

In [17]:
input_seq = tokens.texts_to_sequences([input_text])

In [18]:
pad_input_seq  = sequence.pad_sequences(input_seq , maxlen=100)

In [19]:
pad_input_seq

array([[   2,   73,  252,   30,  255,  331,    1,   60,   48,  101,   22,
           2,  664,   84,   32,   22,  249,    2,  252,   30,  152,   33,
         157,  109,   79,   64,  200,   44,  228,   10,   18,   73,   40,
           2,   17,   11,   58,  111,  589,   84,   11,   58,  377,  907,
         422,    5,  359,  442,   10,   18,    6,    3,  596,    8,   73,
         124,   44,   56,    1,  314,  297,   67,   48,   17,    1, 1676,
           7,  839,    6,  142,   28,   14,  907,   64,  104,   17,   32,
         355,    5,   25,  217,   10,   18, 1033,    6,   20,   56,    1,
         975,  750,    6,  124,   70,   10,   18,   84,   41,   29,  218,
         138]], dtype=int32)

In [21]:
# 測試將影評數字串還原為文字串
pad_input_text = tokens.sequences_to_texts(pad_input_seq)

In [22]:
pad_input_text

["and been played by anyone else the only good characters are and mostly because they are fun and played by actors who new life into their original if anything this film been about and but that would never happen because that would mean disney couldn't to kids overall this film is a complete it been better if even the special effects were good but the cgi in particular is i'm all for disney their films but they need to be interesting this film sadly is not even the christmas sequel is better than this film because it's at least something"]

# 5. 輸入影評(數字串，長度為100)進行情意預測

In [23]:
predict_result=model.predict(pad_input_seq)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step


In [24]:
predict_result

array([[0.02290325]], dtype=float32)

In [25]:
predict_result[0][0]

np.float32(0.022903254)

In [26]:
SentimentDict={1:'正面的',0:'負面的'}

In [27]:
SentimentDict[round(predict_result[0][0])]

'負面的'

# 6. 建立predict_review函數

In [28]:
def predict_review(input_text):
  # 轉換影評文字串為數字串，並截長或補短，使數字串長度為100
  input_seq = tokens.texts_to_sequences([input_text])
  pad_input_seq  = sequence.pad_sequences(input_seq , maxlen=100)
  predict_result = model.predict(pad_input_seq)

  SentimentDict = {1: '正面的', 0: '負面的'}
  return SentimentDict[round(predict_result[0][0])]

prediction = predict_review(input_text)
print(f"這篇影評的情意是: {prediction}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
這篇影評的情意是: 負面的


# 7. 影評情意預測

In [30]:
# URL：https://www.imdb.com/title/tt9603208/reviews/?item=rw10517261&ref_=ext_shr_lnk

input_text_1 = '''
Ignore the critics. This was an absolutely fun and super entertaining movie. It was long but never dragged. In fact from the moment I sat in my seat I was at the cinema for over 3 hours. But it never felt like that.

The climax with the bi-planes was superb and one of the best and most exciting action scenes I've ever seen.

For me, this was one of the best Mission Impossible films. Loved it. Have no idea why some of the critics have panned it. Then again what do they know? It's all about having fun on the big screen and this is exactly what Tom Cruise does best. He's an absolute master in it. Highly recommend!
'''

In [31]:
predict_review(input_text_1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


'正面的'

In [34]:
# https://www.imdb.com/title/tt30840798/reviews/?item=rw10530199&ref_=ext_shr_lnk

input_text_2 = '''
...but I just can't hack it anymore. His last three films have all been increasingly tedious. Best I can describe them is as a slog. Yes there is whimsy and great camera work and attention to detail, but the soul (always so present in his early work) is entirely gone. I'm really hoping he has a return to form and brings us something close to the mastery of what came before, or his last great film Grand Budapest, but I fear he is only going further down his ornate, soulless rabbit hole. It has its moments, but if you have felt, as I have, that is past few films have been a bit of a chore, you will be itching for the end credits.
'''

In [35]:
predict_review(input_text_2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


'正面的'

In [36]:
# https://www.imdb.com/title/tt1179056/reviews/?item=rw2243185&ref_=ext_shr_lnk

input_text3 = '''
This movie was awful. The script was weak. It was a sad knock off of the original Nightmare with less special effects. I see why Robert Englund wasn't in this movie. It wasn't worth it. I'm sooooo disappointed. I yawned more than anything and I wasn't scared not once. I really wanted this to be a good remake, but it wasn't. The special effects were 1980s at best. The acting was pretty good, but the script lacked originality and depth. The twist was stupid. I felt like I was watching the original Freddy only unscary, unfunny, and uninteresting. I expected a much better production with today's technology. It was like eating spam when you think you'll have steak. It was taking a cruise and never leaving the port. It was a let down. This movie is awful.

'''

In [37]:
predict_review(input_text3)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


'負面的'

# 8. 用 Gradio 打造情意分析 Web App

In [38]:
import gradio as gr

In [42]:
demo = gr.Interface(
    fn=predict_review,
    inputs=gr.Textbox(lines=5, placeholder="請輸入一段電影評論..."),
    outputs="text",
    title="IMDB 電影評論情緒判斷",
    description="輸入一段影評，我會判斷它是正面還是負面!"
)
demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9db0078cec84df91b7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
